In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix

2023-08-19 18:32:01.386806: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 18:32:01.467561: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 18:32:01.468599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 18:32:03.232178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def download_and_read(url):
    local_file = url.split('/')[-1]
    local_file = local_file.replace("%20", " ")
    p = tf.keras.utils.get_file(local_file, url, extract=True, cache_dir=".")
    local_folder = os.path.join("datasets", local_file.split('.')[0])
    labeled_sentences = []
    for labeled_filename in os.listdir(local_folder):
        if labeled_filename.endswith("_labelled.txt"):
            with open(os.path.join(local_folder, labeled_filename), "r") as f:
                for line in f:
                    sentence, label = line.strip().split('\t')
                    labeled_sentences.append((sentence, label))
    return labeled_sentences

labeled_sentences = download_and_read(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip")
sentences = [s for (s, l) in labeled_sentences]
labels = [int(l) for (s, l) in labeled_sentences]

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size: {:d}".format(vocab_size))
word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size: 5271


In [4]:
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p in [75, 80, 90, 95, 99, 100]])

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [5]:
max_seqlen = 64
# create dataset
sentences_as_ints = tokenizer.texts_to_sequences(sentences)
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(sentences_as_ints, maxlen=max_seqlen)
labels_as_ints = np.array(labels)
dataset = tf.data.Dataset.from_tensor_slices((sentences_as_ints, labels_as_ints))

2023-08-19 18:32:05.732706: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
dataset = dataset.shuffle(10000)
test_size = len(sentences) // 3
val_size = (len(sentences) - test_size) // 10

test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [7]:
class SentimentAnalysisModel(tf.keras.Model):
    def __init__(self, vocab_size, max_seqlen, **kwargs):
        super(SentimentAnalysisModel, self).__init__(**kwargs)
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, max_seqlen)
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(max_seqlen, return_sequences=False))
        self.dense = tf.keras.layers.Dense(64, activation="relu")
        self.out = tf.keras.layers.Dense(1, activation="sigmoid")
        
    def call(self, x):
        x = self.embedding(x)
        x = self.bilstm(x)
        x = self.dense(x)
        x = self.out(x)
        return x

In [8]:
model = SentimentAnalysisModel(vocab_size, max_seqlen)
model.build(input_shape=(batch_size, max_seqlen))
model.summary()

Model: "sentiment_analysis_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  337408    
                                                                 
 bidirectional (Bidirection  multiple                  66048     
 al)                                                             
                                                                 
 dense (Dense)               multiple                  8256      
                                                                 
 dense_1 (Dense)             multiple                  65        
                                                                 
Total params: 411777 (1.57 MB)
Trainable params: 411777 (1.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# compile
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [10]:
# train
data_dir = "./data"
logs_dir = os.path.join("./logs")
best_model_file = os.path.join(data_dir, "best_model.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file,
                                                 save_weights_only=True,
                                                 save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs,
                    validation_data=val_dataset,
                    callbacks=[checkpoint, tensorboard])

Epoch 1/10
29/29 [==============================] - 10s 79ms/step - loss: 0.6918 - accuracy: 0.5400 - val_loss: 0.6822 - val_accuracy: 0.7100
Epoch 2/10
29/29 [==============================] - 1s 28ms/step - loss: 0.6759 - accuracy: 0.7117 - val_loss: 0.5092 - val_accuracy: 0.7600
Epoch 3/10
29/29 [==============================] - 1s 29ms/step - loss: 0.4866 - accuracy: 0.8028 - val_loss: 0.4014 - val_accuracy: 0.8350
Epoch 4/10
29/29 [==============================] - 1s 28ms/step - loss: 0.3287 - accuracy: 0.8817 - val_loss: 0.2056 - val_accuracy: 0.9500
Epoch 5/10
29/29 [==============================] - 1s 40ms/step - loss: 0.2047 - accuracy: 0.9322 - val_loss: 0.1707 - val_accuracy: 0.9300
Epoch 6/10
29/29 [==============================] - 2s 54ms/step - loss: 0.1545 - accuracy: 0.9544 - val_loss: 0.1833 - val_accuracy: 0.9550
Epoch 7/10
29/29 [==============================] - 2s 57ms/step - loss: 0.1089 - accuracy: 0.9650 - val_loss: 0.0843 - val_accuracy: 0.9700
Epoch 8/10
2

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [12]:
%tensorboard --logdir logs

In [14]:
# evaluate with test set
best_model = SentimentAnalysisModel(vocab_size, max_seqlen)
best_model.build(input_shape=(batch_size, max_seqlen))
best_model.load_weights(best_model_file)
best_model.compile(
    loss="binary_crossentropy",
    optimizer="adam", 
    metrics=["accuracy"]
)

In [15]:
test_loss, test_acc = best_model.evaluate(test_dataset)
print("test loss: {:.3f}, test accuracy: {:.3f}".format(test_loss, test_acc))

16/16 [==============================] - 3s 10ms/step - loss: 0.0321 - accuracy: 0.9920
test loss: 0.032, test accuracy: 0.992


In [16]:
# predict on batches
labels, predictions = [], []
idx2word[0] = "PAD"
is_first_batch = True
for test_batch in test_dataset:
    inputs_b, labels_b = test_batch
    pred_batch = best_model.predict(inputs_b)
    predictions.extend([(1 if p > 0.5 else 0) for p in pred_batch])
    labels.extend([l for l in labels_b])
    if is_first_batch:
        for rid in range(inputs_b.shape[0]):
            words = [idx2word[idx] for idx in inputs_b[rid].numpy()]
            words = [w for w in words if w != "PAD"]
            sentence = " ".join(words)
            print("{:d}\t{:d}\t{:s}".format(labels[rid], predictions[rid], sentence))
        is_first_batch = False

print("accuracy score: {:.3f}".format(accuracy_score(labels, predictions)))
print("confusion matrix")
print(confusion_matrix(labels, predictions))

2/2 [==============================] - 1s 6ms/step
1	1	everything about this film is simply incredible
0	0	like the other reviewer said you couldn't pay me to eat at this place again
0	0	do yourself a favor and stay away from this dish
0	0	i as well would've given godfathers zero stars if possible
0	0	the only thing i wasn't too crazy about was their guacamole as i don't like it puréed
1	1	very convenient since we were staying at the mgm
1	1	this product is great it makes working a lot easier i can go to the copier while waiting on hold for something
0	0	i don't know what happened in season five what a mess
1	1	kieslowski never ceases to amaze me
0	0	sprint charges for this service
0	0	i have been in more than a few bars in vegas and do not ever recall being charged for tap water
0	0	the story starts too fast with absolutely no suspense or build up in the slightest
1	1	feelings thoughts gabriel's discomfort during the dance all these intangibles leap to life and come within the viewer'

---